<a href="https://colab.research.google.com/github/bfeijoj/Book-Recommendation/blob/main/book_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import zipfile
from IPython.display import clear_output

In [2]:
# -------------------------------------------------------------------------- Recommend Function ----------------------------------------------------------------------------------------------

def get_recommends(book = ""):

  distance, recommended_books = model.kneighbors(df_user_title.loc[book, :].values.reshape(1, -1))
  recommended = df_user_title.index[recommended_books][0][1:]

  return print(' First Book = ',recommended[0],'\n',
  				'Second Book = ',recommended[1],'\n',
  				'Third Book = ',recommended[2],'\n',
  				'Forth Book = ',recommended[3])

In [3]:
# ------------------------------------------------------------------------- Importing Files -------------------------------------------------------------------------------------------------

!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-12-20 19:39:12--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   163MB/s    in 0.2s    

2021-12-20 19:39:12 (163 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
# ------------------------------------------------------------------------- CSV to DataFrame ------------------------------------------------------------------------------------------------

# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [5]:
# ------------------------------------------------------------------------- Data Management  ------------------------------------------------------------------------------------------------

relevant_users = df_ratings['user'].value_counts() > 200
relevant_users = relevant_users[relevant_users == True].index

df_ratings = df_ratings[df_ratings['user'].isin(relevant_users)]

df_ratings_books = df_ratings.merge(df_books, on = 'isbn')

relevant_books = df_ratings_books['isbn'].value_counts() > 100
relevant_books = relevant_books[relevant_books == True].index

df_ratings_books = df_ratings_books[df_ratings_books['isbn'].isin(relevant_books)]

df_ratings_books.drop_duplicates(['user','isbn'], inplace=True)

df_user_title = df_ratings_books.pivot_table(columns = 'user', index = 'title', values = 'rating')
df_user_title.fillna(0, inplace = True)

df_user_title_sparse = csr_matrix(df_user_title)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [6]:
# ------------------------------------------------------------------------- Machine Learning Model --------------------------------------------------------------------------------

model = NearestNeighbors(metric="cosine",algorithm="brute", p=2)
model.fit(df_user_title_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [10]:
# -------------------------------------------------------------------------- Recommendations ------------------------------------------------------------------------------------------------

get_recommends(book = "A Walk to Remember")

 First Book =  The Notebook 
 Second Book =  Circle of Friends 
 Third Book =  Bridget Jones's Diary 
 Forth Book =  The Bridges of Madison County


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  
